
Congrats....again! Based off the amazing work you did on the classification model, you've been promoted to chief of all data scientists in Hollywood, your a total celebrity....among data scientists :).  

Your boss, head of the studio, has now asked you to build a model to predict gross revenue in order to help them decide which movies to invest in.

Once again, you would like to be able to explain the model to mere mortals but need a fairly robust and flexible approach so you've chosen to use decision trees to get started. 

In doing so...you leverage work you've done in the past to get the job done....you're a data scientist after all! 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#1. Load the data
#Sometimes need to set the working directory back out of a folder that we create a file in

#import os
#os.listdir()
#print(os.getcwd())
#os.chdir('c:\\Users\\Brian Wright\\Documents\\3001Python\\DS-3001')

movie_metadata = pd.read_csv("/workspaces/DS-3021/data/movie_metadata.csv")
movie_metadata.head()



,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


# 2 Ensure all the variables are classified correctly including the target variable and collapse factor variables as needed.

# 3 Check for missing variables and correct as needed.

In [3]:
def preprocess_data_reg(df):
    df = df.dropna(subset=["gross"])
    drop_columns = ["actor_1_name", "actor_3_name", "movie_title", "movie_imdb_link", 
                    "director_name", "director_facebook_likes", "actor_1_facebook_likes", 
                    "actor_2_facebook_likes", "actor_3_facebook_likes", "actor_2_name", 
                    "aspect_ratio", "facenumber_in_poster", "num_critic_for_reviews", 
                    "num_user_for_reviews", "movie_facebook_likes", "cast_total_facebook_likes", 
                    "gross"]  

    df["content_rating"] = df["content_rating"].replace({"Unrated": "Not Rated", "Not Rated": "Not Rated"}).apply(
        lambda x: x if x in ["R", "PG-13", "PG", "Not Rated"] else "Other")
    df["content_rating"] = pd.factorize(df["content_rating"])[0]

    df["color"] = df["color"].apply(lambda x: 1 if x == "Color" else 0)

    df["plot_keywords"] = df["plot_keywords"].apply(
        lambda x: x.split("|")[0] if isinstance(x, str) and "|" in x else x)
    df["plot_keywords"] = pd.factorize(df["plot_keywords"])[0]

    df["genres"] = df["genres"].apply(
        lambda x: x.split("|")[0] if isinstance(x, str) and "|" in x else x)
    df["genres"] = pd.factorize(df["genres"])[0]

    df["country"] = df["country"].apply(lambda x: x if x in ["USA", "UK", "France", "Canada", "Germany"] else "Other")
    df["country"] = pd.factorize(df["country"])[0]

    df["language"] = df["language"].apply(lambda x: x if x in ["English", "French", "Spanish"] else "Other")
    df["language"] = pd.factorize(df["language"])[0]

    df.dropna(inplace=True)

    X = df.drop(columns=drop_columns, errors="ignore")
    y = df["gross"]

    return X, y

X, y = preprocess_data_reg(movie_metadata)


/tmp/ipykernel_508/1867913668.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["content_rating"] = df["content_rating"].replace({"Unrated": "Not Rated", "Not Rated": "Not Rated"}).apply(
/tmp/ipykernel_508/1867913668.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["content_rating"] = pd.factorize(df["content_rating"])[0]
/tmp/ipykernel_508/1867913668.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

# 4 Guess what, you don't need to scale the data, because DTs don't require this to be done, they make local greedy decisions...keeps getting easier, go to the next step.

# 5 Determine the variance of the target variable, is it normally distributed? If not, you may want to transform the target variable. Why might this be important?

In [4]:
baserate = np.mean(y)
print(f"Base rate: {baserate}")

Base rate: 52005758.516443044


# 6 Split your data into test, tune, and train. (80/10/10)

In [5]:
from sklearn.model_selection import train_test_split
X_train_tune, X_test, y_train_tune, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_tune, y_train, y_tune = train_test_split(X_train_tune, y_train_tune, test_size=0.2, random_state=42)

print(f"Train set size: {X_train.shape[0]}")
print(f"Tune set size: {X_tune.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")


Train set size: 2432
Tune set size: 608
Test set size: 761


# 7 Create the kfold object for cross validation.

In [6]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 8 Create the scoring metric (several measures) you will use to evaluate your model and choose a hyperparamater to optimize.

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#first we train the model on the training set
model = DecisionTreeRegressor(max_depth=5, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"R² Score: {r2:.2f}")

evaluate_model(y_test, y_pred)


Mean Squared Error (MSE): 2528758175042046.50
Mean Absolute Error (MAE): 29533171.53
R² Score: 0.58


# 9 Build the regression tree object. 

In [8]:
from sklearn.tree import DecisionTreeRegressor
reg_tree = DecisionTreeRegressor(max_depth=5, random_state=42)

# 10 Fit the model to the training data.

In [9]:
#fit the model
reg_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, random_state=42)

# 11 Explore the results of the model. Print the output of the grid search. What is the best parameter and score?

In [10]:
from sklearn.model_selection import GridSearchCV
reg_tree = DecisionTreeRegressor(random_state=42)

param_grid = {'max_depth': [None, 5, 10, 15, 20]}
grid_search = GridSearchCV(estimator=reg_tree, 
                           param_grid=param_grid, 
                           scoring='neg_mean_squared_error',  # or 'r2', etc.
                           cv=kfold,
                           return_train_score=True)

# Fit on training data
grid_search.fit(X_train, y_train)


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [None, 5, 10, 15, 20]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [11]:
print("Best max_depth:", grid_search.best_params_['max_depth'])
print("Best CV Score (MSE):", -grid_search.best_score_)


Best max_depth: 5
Best CV Score (MSE): 2311195817653755.5


# 12 Which variables appear to be contributing the most (variable importance)

In [12]:
best_model = grid_search.best_estimator_

#get the feature importances
importances = best_model.feature_importances_

importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df.head(10))


            Feature  Importance
3   num_voted_users    0.622640
7    content_rating    0.172813
8            budget    0.104896
9        title_year    0.037271
2            genres    0.027677
6           country    0.013035
4     plot_keywords    0.011386
10       imdb_score    0.010281
0             color    0.000000
1          duration    0.000000


# 13 Create a model object using the best model hyperparameter value from the trained regression tree.

In [13]:
from sklearn.tree import DecisionTreeRegressor
best_depth = grid_search.best_params_['max_depth']
best_reg_tree = DecisionTreeRegressor(max_depth=best_depth, random_state=42)


# 14 Using the best model predict on the tune data and print out the results.

In [14]:
best_reg_tree.fit(X_train, y_train)
y_tune_pred = best_reg_tree.predict(X_tune)


In [16]:
y_tune_pred = best_reg_tree.predict(X_tune)
results_df = pd.DataFrame({
    'Actual Gross': y_tune.values,
    'Predicted Gross': y_tune_pred
})

print(results_df.head(10))


   Actual Gross  Predicted Gross
0   186739919.0     1.252142e+08
1      700000.0     2.234254e+07
2    34793160.0     2.234254e+07
3    19781879.0     6.098682e+07
4    17683670.0     3.620270e+07
5    25093607.0     4.054041e+07
6      568695.0     8.488036e+06
7    53133888.0     3.620270e+07
8     2428883.0     2.234254e+07
9   144156464.0     6.988103e+07


# 15 How does the model perform on the tune data as compared to the training data?

In [18]:
#predict the train set
y_train_pred = best_reg_tree.predict(X_train)
print("Training Performance:")
evaluate_model(y_train, y_train_pred)


Training Performance:
Mean Squared Error (MSE): 1366921444412778.25
Mean Absolute Error (MAE): 24055771.64
R² Score: 0.70


In [19]:
#predict the tune set
y_tune_pred = best_reg_tree.predict(X_tune)
print("Tuning Performance:")
evaluate_model(y_tune, y_tune_pred)


Tuning Performance:
Mean Squared Error (MSE): 2132923314355706.00
Mean Absolute Error (MAE): 27322686.91
R² Score: 0.54


# 16 What five movies are predicted to have the lowest gross revenue from the tune set?

In [20]:
tune_results_df = pd.DataFrame({
    'Predicted Gross': y_tune_pred,
    'Actual Gross': y_tune.values
}, index=X_tune.index)  
tune_results_df['Movie Title'] = movie_metadata.loc[X_tune.index, 'movie_title']
#do the bottom 5
lowest_predicted = tune_results_df.sort_values(by='Predicted Gross').head(5)

print(lowest_predicted[['Movie Title', 'Predicted Gross', 'Actual Gross']])


           Movie Title  Predicted Gross  Actual Gross
3529  The Heart of Me      1.234206e+06      196067.0
3195           Malone      1.234206e+06     3060858.0
4827      Tumbleweeds      1.234206e+06     1281176.0
4435          Trucker      1.234206e+06       52166.0
3319     Undiscovered      1.234206e+06     1046166.0


# 17 As a data scientist you are always looking to improve your results. What are some ideas you have to improve the model? Have you ask...WHAT IF?!

#### 1. what if i try using log on the target 
by using a natual log to the gross revenue column before trainng model and then exponentiating the predictions back using np.expm1() to interpret results in the original scale. because the reveue is very skewed so this method helps to compress the outliers and highlight smaller trends.
#### 2. what if i try a random forest instead of a single decision tree
by replacing the DecisionTreeRegressor with a RandomForestRegressor using the same hyperparameter tuning setup. because the decision trees are prone to overfitting and they memorize the training set rather than generalizing. A Random Forest averages many trees trained on random subsets, which usually improves performance and reduces variance.


# 18 Experiment using your what if ideas (two is fine). Did the model improve? If so, how much? If not, what did you learn? (You don't need to include all the "what if" code just speak to what you tried and what you learned).

In [21]:
def preprocess_data_loggross(df):
    df = df.dropna(subset=["gross"])

    drop_columns = ["actor_1_name", "actor_3_name", "movie_title", "movie_imdb_link", 
                    "director_name", "director_facebook_likes", "actor_1_facebook_likes", 
                    "actor_2_facebook_likes", "actor_3_facebook_likes", "actor_2_name", 
                    "aspect_ratio", "facenumber_in_poster", "num_critic_for_reviews", 
                    "num_user_for_reviews", "movie_facebook_likes", "cast_total_facebook_likes", 
                    "gross"]

    df["content_rating"] = df["content_rating"].replace({"Unrated": "Not Rated", "Not Rated": "Not Rated"}).apply(
        lambda x: x if x in ["R", "PG-13", "PG", "Not Rated"] else "Other")
    df["content_rating"] = pd.factorize(df["content_rating"])[0]

    df["color"] = df["color"].apply(lambda x: 1 if x == "Color" else 0)

    df["plot_keywords"] = df["plot_keywords"].apply(
        lambda x: x.split("|")[0] if isinstance(x, str) and "|" in x else x)
    df["plot_keywords"] = pd.factorize(df["plot_keywords"])[0]

    df["genres"] = df["genres"].apply(
        lambda x: x.split("|")[0] if isinstance(x, str) and "|" in x else x)
    df["genres"] = pd.factorize(df["genres"])[0]

    df["country"] = df["country"].apply(lambda x: x if x in ["USA", "UK", "France", "Canada", "Germany"] else "Other")
    df["country"] = pd.factorize(df["country"])[0]

    df["language"] = df["language"].apply(lambda x: x if x in ["English", "French", "Spanish"] else "Other")
    df["language"] = pd.factorize(df["language"])[0]

    df.dropna(inplace=True)

    X = df.drop(columns=drop_columns, errors="ignore")
    y = np.log1p(df["gross"]) 

    return X, y

X_log, y_log = preprocess_data_loggross(movie_metadata)


/tmp/ipykernel_508/3935326486.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["content_rating"] = df["content_rating"].replace({"Unrated": "Not Rated", "Not Rated": "Not Rated"}).apply(
/tmp/ipykernel_508/3935326486.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["content_rating"] = pd.factorize(df["content_rating"])[0]
/tmp/ipykernel_508/3935326486.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [22]:
from sklearn.model_selection import train_test_split

X_train_log, X_temp_log, y_train_log, y_temp_log = train_test_split(X_log, y_log, test_size=0.2, random_state=42)
X_tune_log, X_test_log, y_tune_log, y_test_log = train_test_split(X_temp_log, y_temp_log, test_size=0.5, random_state=42)
log_model = DecisionTreeRegressor(max_depth=5, random_state=42)
log_model.fit(X_train_log, y_train_log)
y_tune_pred_log = np.expm1(log_model.predict(X_tune_log))  
y_tune_actual = np.expm1(y_tune_log)  


In [23]:
evaluate_model(y_tune_actual, y_tune_pred_log)

Mean Squared Error (MSE): 3503225520409479.50
Mean Absolute Error (MAE): 31607797.32
R² Score: 0.48


In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf_model = RandomForestRegressor(random_state=42)
param_grid_rf = {
    'max_depth': [5, 10, 15, None],
    'n_estimators': [50, 100]
}

grid_rf = GridSearchCV(estimator=rf_model,
                       param_grid=param_grid_rf,
                       scoring='neg_mean_squared_error',
                       cv=kfold,
                       return_train_score=True)

grid_rf.fit(X_train, y_train)
best_rf = grid_rf.best_estimator_
y_tune_pred_rf = best_rf.predict(X_tune)
evaluate_model(y_tune, y_tune_pred_rf)

Mean Squared Error (MSE): 1401668689650583.25
Mean Absolute Error (MAE): 22605430.53
R² Score: 0.70


## ^ random forest is doing a lot beter than the original decision tree

In [25]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

reg_tree = DecisionTreeRegressor(random_state=42)
param_grid_expanded = {
    'max_depth': [5, 10, 15],
    'min_samples_leaf': [1, 5, 10, 20]
}

grid_search_expanded = GridSearchCV(estimator=reg_tree,
                                    param_grid=param_grid_expanded,
                                    scoring='neg_mean_squared_error',
                                    cv=kfold,
                                    return_train_score=True)

grid_search_expanded.fit(X_train, y_train)
best_model_expanded = grid_search_expanded.best_estimator_
y_tune_pred_expanded = best_model_expanded.predict(X_tune)

print("Best Parameters:", grid_search_expanded.best_params_)
evaluate_model(y_tune, y_tune_pred_expanded)


Best Parameters: {'max_depth': 15, 'min_samples_leaf': 10}
Mean Squared Error (MSE): 2179091339267393.75
Mean Absolute Error (MAE): 26591862.98
R² Score: 0.53


## ^ tuning another hyperparameter didn't improve a lot but i kept it from overfitting too much

# 19 Use the best model to predict on the test data and print out the evaluation results.

In [26]:
y_test_pred_rf = best_rf.predict(X_test)
print("Final Evaluation on Test Set:")
evaluate_model(y_test, y_test_pred_rf)


Final Evaluation on Test Set:
Mean Squared Error (MSE): 1581157403036303.75
Mean Absolute Error (MAE): 24128826.74
R² Score: 0.74


# 20 Does the model perform well on the test data when compared to the training and tune data? Why or why not?

##### The model performs well on the test data, showing consistent performance compared to both the training and tune sets. If the evaluation metrics on the test set (such as R² score and MAE) are close to those observed on the tune set, it suggests that the model generalizes effectively and hasn’t overfit the training data. This consistency indicates that the Random Forest model was able to learn meaningful patterns in the data without memorizing it. On the other hand, if the test performance dropped significantly, it would suggest potential overfitting. 

# 21 Where are the errors occurring in your predictions? Are they systematic or random? In either case, what are the implications of this?

#### The errors in the predictions is somewhat systematic, especially for movies with extremely high or low gross revenues. The model tends to underestimate blockbusters and overestimate lower-performing films, which suggests it struggles with capturing the full range of variability in the target variable. While the model performs well on average, it may not be reliable for forecasting extreme cases, like identifying breakout hits or box office flops. In practice, this could affect decision-making for high-stakes investments or risk assessments in the film industry, emphasizing the need for additional data or more advanced models that can better handle extreme values.

# 22 Summarize what you learned along the way and make recommendations on how this could be used moving forward, being careful not to over promise.

Throughout this project, I learned how to build, evaluate, and improve a regression model for predicting movie gross revenue using decision trees and random forests. By experimenting with different hyperparameters and model types, I saw how model choice and tuning can significantly impact performance and generalization. While the final model performs well on unseen data, its predictions should be used as directional insights rather than exact forecasts, especially for outlier cases like blockbuster hits or box office flops.